In [22]:
#Kütüphaneleri yükle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

#Python'da uyarıları kapatalım
import warnings
warnings.filterwarnings("ignore")

In [23]:
#veriyi içeri aktaralım
veri=pd.read_csv("olimpiyatlar.csv")
veri.head()

,Name,Gender,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Edgar Lindenau Aabye,M,34.0,182.48,95.62,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
1,Arvo Ossian Aaltonen,M,30.0,182.01,76.69,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
2,Arvo Ossian Aaltonen,M,30.0,177.00,75,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
3,Juhamatti Tapio Aaltonen,M,28.0,184.00,85,Finland,FIN,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
4,Paavo Johannes Aaltonen,M,28.0,175.00,64,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze


In [24]:
veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39783 entries, 0 to 39782
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    39783 non-null  object 
 1   Gender  39783 non-null  object 
 2   Age     39783 non-null  float64
 3   Height  39783 non-null  float64
 4   Weight  39783 non-null  object 
 5   Team    39783 non-null  object 
 6   NOC     39783 non-null  object 
 7   Year    39783 non-null  int64  
 8   Season  39783 non-null  object 
 9   City    39783 non-null  object 
 10  Sport   39783 non-null  object 
 11  Event   39783 non-null  object 
 12  Medal   39783 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 3.9+ MB


In [25]:
veri.columns

Index(['Name', 'Gender', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Year',
       'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [26]:
#sütun isimlerini değiştirelim
veri.rename(columns={'Name'  : 'isim',
                     'Gender': 'cinsiyet',
                     'Age'   : 'yas',
                     'Height': 'boy',
                     'Weight': 'kilo',
                     'Team'  : 'takim',
                     'NOC'   : 'uok',
                     'Year'  : 'yil' ,                    
                     'Season': 'sezon',
                     'City'  : 'sehir',
                     'Sport' : 'spor',
                     'Event' : 'etkinlik',
                     'Medal' : 'madalya'}, inplace=True)
veri.head(2)

,isim,cinsiyet,yas,boy,kilo,takim,uok,yil,sezon,sehir,spor,etkinlik,madalya
0,Edgar Lindenau Aabye,M,34.0,182.48,95.62,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
1,Arvo Ossian Aaltonen,M,30.0,182.01,76.69,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze


In [27]:
#drop fonksiyonu ile id ve oyunlar çıkartılıyor fakat benim veri setinde zaten yoklar
#veri=veri.drop(["id","oyunlar"],axis=1) 

In [28]:
#olmayan veriler NaN (not a number) anlamına gelir bunlar kayıp veridir
veri.head()

,isim,cinsiyet,yas,boy,kilo,takim,uok,yil,sezon,sehir,spor,etkinlik,madalya
0,Edgar Lindenau Aabye,M,34.0,182.48,95.62,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
1,Arvo Ossian Aaltonen,M,30.0,182.01,76.69,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
2,Arvo Ossian Aaltonen,M,30.0,177.00,75,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
3,Juhamatti Tapio Aaltonen,M,28.0,184.00,85,Finland,FIN,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
4,Paavo Johannes Aaltonen,M,28.0,175.00,64,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze


In [32]:
essiz_etkinlik=pd.unique(veri.etkinlik)
print("Eşşiz etkinlik sayısı: {}".format(len(essiz_etkinlik)))
essiz_etkinlik[:10]

Eşşiz etkinlik sayısı: 756


array(["Tug-Of-War Men's Tug-Of-War",
       "Swimming Men's 200 metres Breaststroke",
       "Swimming Men's 400 metres Breaststroke",
       "Ice Hockey Men's Ice Hockey",
       "Gymnastics Men's Individual All-Around",
       "Gymnastics Men's Team All-Around", "Gymnastics Men's Horse Vault",
       "Gymnastics Men's Pommelled Horse", "Alpine Skiing Men's Super G",
       "Alpine Skiing Men's Giant Slalom"], dtype=object)

In [34]:
# her bir etkinliği iteratif olarak dolaştırıyoruz
# etkinlik özelinde boy ve kilo ortalamalarını hesapla
# etkinlik özelinde kayıp boy ve kilo değerlerini etkinlik ortalamasına eşitle

veri_gecici=veri.copy() # gerçek veriyi bozmamak için veriyi kopyaladık
boy_kilo_liste=["boy","kilo"]

for e in essiz_etkinlik: # Liste içerisinde dolaş

    # Etkinlik filtresi oluşturalım
    etkinlik_filtre = veri_gecici.etkinlik == e
    # Veriyi etkinliğe göre filtreyeleim
    veri_filtreli = veri_gecici[etkinlik_filtre]
    
    # Boy ve kilo için etkinlik özelinde ortalamalarını hesaplayalım
    for s in boy_kilo_liste:
        ortalama = np.round(np.mean(veri_filtreli[s]),2)
        if ~np.isnan(ortalama): #  Eğer etkinlik özelinde ortalama varsa
            veri_filtreli[s] = veri_filtreli[s].fillna(ortalama)
        else: # Eğer etkinlik özelinde ortalama varsa ortalamayı hesapla
            tum_veri_ortalamasi = np.round(np.mean(veri[s],2))
            veri_filtreli[s] = veri_filtreli[s].fillna(tum_veri_ortalamasi)
            
    #etkinlik özelinde kauıp değerleri doldurulmuş olan veriyi, veri_gecici
    veri_gecici[etkinlik_filtre] = veri_filtreli
    
#kayıp değerleri giderilmiş olan veriyi gerçek veriye eşitleyelim
veri = veri_gecici.copy()
veri.info()


TypeError: Could not convert 95.6295.629995.6295.6295.6295.6295.6295.6210095.6295.6295.628395.628195.6295.6295.6295.6295.6295.6295.6295.6295.629995.628995.6295.6295.6295.6295.6295.629595.6295.6295.6210010010095.629395.6295.6295.6295.6295.62779695.627595.6295.6295.628895.629495.6295.6210410010010095.6295.6295.6295.6295.6295.629895.6295.6295.6295.628095.6295.6295.6295.6295.6295.628295.6295.6295.6295.6210410410495.629595.6295.6295.6295.6210895.6295.6295.6295.6295.6295.6295.6295.6295.6295.6295.6295.6295.628295.6295.6295.6295.62 to numeric